In [ ]:
!pip install transformers torchaudio datasets
!pip install nnAudio

In [ ]:
import os

import numpy as np
import torch
import torchaudio
from sklearn.decomposition import PCA
from torch import nn
from transformers import AutoModel, Wav2Vec2FeatureExtractor
from datasets import load_dataset
import torchaudio.transforms as T

class CustomResampler(nn.Module):
    def __init__(self, orig_freq, new_freq):
        super(CustomResampler, self).__init__()
        self.orig_freq = orig_freq
        self.new_freq = new_freq
        self.resampler = T.Resample(orig_freq, new_freq)

    def forward(self, waveform):
        resampled = self.resampler(waveform)
        return resampled

def save_waveform(tensor, filename):
    torchaudio.save(filename, tensor, sample_rate=resample_rate)

def create_waveform_from_hidden_state(hidden_state, sample_rate):
    pca = PCA(n_components=2)
    reduced_dim_data = pca.fit_transform(hidden_state.T).T
    normalized_waveform = (reduced_dim_data - reduced_dim_data.min()) / (reduced_dim_data.max() - reduced_dim_data.min())
    return torch.tensor(normalized_waveform, dtype=torch.float32)

model = AutoModel.from_pretrained("m-a-p/MERT-v1-95M", trust_remote_code=True)
processor = Wav2Vec2FeatureExtractor.from_pretrained("m-a-p/MERT-v1-95M", trust_remote_code=True)

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

resample_rate = processor.sampling_rate
if resample_rate != sampling_rate:
    print(f'Setting rate from {sampling_rate} to {resample_rate}')
    resampler = CustomResampler(sampling_rate, resample_rate)
else:
    resampler = None

if resampler is None:
    input_audio = dataset[0]["audio"]["array"]
else:
    input_audio = resampler(torch.from_numpy(dataset[0]["audio"]["array"]).float())

inputs = processor(input_audio.squeeze(), sampling_rate=resample_rate, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True)

all_layer_hidden_states = torch.stack(outputs.hidden_states).squeeze()

output_dir = "waveforms"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i, hidden_state in enumerate(all_layer_hidden_states):
    waveform_filename = os.path.join(output_dir, f"hidden_state_{i}.wav")
    waveform = create_waveform_from_hidden_state(hidden_state.detach().numpy(), resample_rate)
    save_waveform(waveform, waveform_filename)

print(f"Waveform files saved in {output_dir}")